# Covering Radius of Hermitian Codes


## Imports

Here we import the neccesary packages

In [22]:
import numpy as np
import pandas as pd
import pickle
import datetime as dt
import math

## Hermitian Codes

Implementations for obtaining the generator and parity check matrix of the Hermitian Code $H_m$ over $\mathbb{F}_{q^2}$. 

Definitions for the Hermitian code can be found in **link paper** 

Details for the SAGE methods can be found in **link sage doc**

In [23]:
def HermitianCodeGenMatrix(m,q):
    #The code is over GF(q^2)
    F=GF(q^2,'a')
    
    #we define the rational function field
    K.<x> = FunctionField(F); _.<Y> = K[]
    #and the hermitian extension [x^(q+1)-y^q-y=0 or x^(q+1)=y^q+y]
    H.<y>=K.extension(x^(q+1)-Y^q-Y)
    
    #then, we get a list of the rational places
    L=H.places_finite(1) #q^3 rational places
    assert(len(L)==q^3)
    #and the infinite place
    Inf=H.places_infinite()
    assert(len(Inf)==1) #there should be only one for the hermitian case 
    P_Inf=Inf[0]
    
    #Then we need to get the basis for the Riemann-Roch space of m*P_Inf
    Basis=(m*P_Inf).basis_function_space()
    
    #Finally , we evaluate on the points to get the generator matrix
    n=len(L) # n= code length, w number of points
    k=len(Basis) # k= dimension, it is the length of the basis
    GeneratorMatrix=matrix(F,k,n, lambda i,j: Basis[i].evaluate(L[j])) #G_ij=f_i(P_j)
    return GeneratorMatrix.rref() #and we return it in reduced-row-echelon form 

In [24]:
def HermitianCode(m,q):
    #Use the generator matrix to form the SAGE codes.LinearCode object 
    return codes.LinearCode(HermitianCodeGenMatrix(m,q))

In [25]:
def HermitianCodeCheckMatrix(m,q):
    #use the SAGE "parity_check_matrix()" method to obtain the parity check matrix from the Hermitain Code
    return HermitianCode(m,q).parity_check_matrix()

## Syndrome Calculation

To compute the Covering Radius, we will compute a table of the syndromes of words of small weight starting from 1 and stopping when the number of total syndromes equals $(q^2)^{n-k}$ (or when it gets too computationally expensive to continue), as stated in section 9 of the paper ***link paper***

SAGE also implents this method when you call the ***insert link to covering radius method*** found in ***link sagemth doc***

However, because these vectors are sparse, we can do some optimizations to eliminate some unnecsary multiplications and additions in the base field. We provide some visualization of the optimizations performed below, as well as code snippets, and an exceutable version of the code 

For example, in our work, we fill out syndrome tables of Herimitian codes of length 27 over the finite field of size 9. To calculate a syndrome of a word of weight 2, only 2 coordinates will be non zero, and the other 25 will be 0. If we were to calculate the syndrome "naively" (that is, by simply multiplying the parity check matrix by the word) this will do a lot of unncecsary computations. 

Let's take the $[27,23,3]_9$ Hermitian code $H_{25}$ over $\mathbb{F}_9$, and choose a particular vector of weight 2. 

```
#Set the parameters
m=25
q=3

#this gets the parity check matrix of the Hermitian code H_25, as defined above
CheckMatrix=HermitianCodeCheckMatrix(m,q)

# this defines the Galois field with 9 elements, and sets a as the generating element
BaseField.<a>=GF(9) 

# Choose a vector of weight 2
word=vector(BaseField,[0,0,0,0,0,0,0,0,0,           
                   0,0,0,0,0,1,0,0,0,
                   0,0,0,0,2*a+1,0,0,0,0])  

#We can get the syndrome by multiplying the parity check matrix by the word (using right multiplication) 
Syndrome=CheckMatrix*word
print(Syndrome)

#And to check the running time, we use the timeit module with 1000 iterations
timeit("CheckMatrix*word", number=1000)

```

**Executable version below**

In [26]:
#Set the parameters
m=25
q=3

#this gets the parity check matrix of the Hermitian code H_22, as defined above
CheckMatrix=HermitianCodeCheckMatrix(m,q)

# this defines the Galois field with 9 elements, and sets a as the generating element
BaseField.<a>=GF(9) 

# Choose a vector of weight 2
word=vector(BaseField,[0,0,0,0,0,0,0,0,0,           
                   0,0,0,0,0,1,0,0,0,
                   0,0,0,0,2*a+1,0,0,0,0])  

#We can get the syndrome by multiplying the parity check matrix by the word (using right multiplication) 
Syndrome=CheckMatrix*word
print(Syndrome)

#And to check the running time, we use the timeit module with 1000 iterations
timeit("CheckMatrix*word", number=1000)

(0, 2*a + 1, 1, 0)


1000 loops, best of 3: 24.9 μs per loop

We can instead represent this vector $\mathbf{x}$ by taking note of two things: its non-zero positions (which would be positions 15 and 23) and its corresponding coordinates ( 1 and $2a+1$ ) and using this select only the relevant columns of the parity check matrix $\mathbf{H}_m$ (the ones corresponding to the positions) and multiplying them by the relevant coeficients. This is because 

$\mathbf{H}_m  \mathbf{x} = \sum\limits_{i=1}^{n}{\mathbf{H}_m^{(i)} \cdot \mathbf{x}}=\sum\limits_{i \ \mid x_i \neq 0}{\mathbf{H}_m^{(i)}x_i}$

In this case, it would be like this2:

```
# these are the non-zero positions
positions=(15,23)

# and the corresponding non zero coeficients
coeficients=vector(BaseField,[1,2*a+1])

# to make the alternate Syndrome calculation, 
# we simply select the columns of the parity check matrix corresponding to the non-zero positions
# and multiply by the corresponding non-zero coeficients
# (because we are coutning the positions starting from 1, we must substarct 1 from the index of the columns since SAGE is 0 based)
AlternateSyndrome=CheckMatrix.columns()[positions[0]-1]*coeficients[0]+ CheckMatrix.columns()[positions[1]-1]*coeficients[1]
print(AlternateSyndrome)

# and we make sure that these do in fact give the same result 
assert(AlternateSyndrome==Syndrome)

# finally, we check the running time using the timeit module
timeit("AlternateSyndrome=CheckMatrix.columns()[positions[0]-1]*coeficients[0]+ CheckMatrix.columns()[positions[1]-1]*coeficients[1]", number=1000)
```

**Excecutable version below**

In [27]:
# these are the non-zero positions
positions=(15,23)

# and the corresponding non zero coeficients
coeficients=vector(BaseField,[1,2*a+1])

# to make the alternate Syndrome calculation, 
# we simply select the columns of the parity check matrix corresponding to the non-zero positions
# and multiply by the corresponding non-zero coeficients
# (because we are coutning the positions starting from 1, we must substarct 1 from the index of the columns since SAGE is 0 based)
AlternateSyndrome=CheckMatrix.columns()[positions[0]-1]*coeficients[0]+ CheckMatrix.columns()[positions[1]-1]*coeficients[1]
print(AlternateSyndrome)

# and we make sure that these do in fact give the same result 
assert(AlternateSyndrome==Syndrome)

# finally, we check the running time using the timeit module
timeit("AlternateSyndrome=CheckMatrix.columns()[positions[0]-1]*coeficients[0]+ CheckMatrix.columns()[positions[1]-1]*coeficients[1]", number=1000)

(0, 2*a + 1, 1, 0)


1000 loops, best of 3: 1.68 μs per loop

As we can see from the executable versions, both methods give the same results, but the second method is much faster. 

To this end, we define the following function to perform this calculation in general

In [28]:
def SyndromeCalculation(CheckMatrix, coefficients, positions):
    # make sure we have as many coeficients as positions
    assert(len(coefficients)==len(positions))
    
    # Take the relevant columns of the check matrix (remembering to substract 1 as SAGE is 0 based)
    # and multiply by the corresponding coeficient and store them in a vector
    result=[CheckMatrix.columns()[positions[i]-1]*coefficients[i] for i in range(len(positions))]
    
    # Finally, add them up to obtain the syndrome
    Syndrome=sum(result)
    
    
    return (Syndrome)


### Possitions and Coefficients

Now that we have a way to calculate the syndrome using **idk** 

In [29]:
def GetPossibleWordPositions(length, weight):
    # First, get the list of all subsets of [1,2,3,...,length] that have 'weight' elements
    # This represents all posibilities of non-zero-positions of a vector of length='length' and weight='weight'
    SubSets=Subsets(list(range(1,length+1)),weight).list()
    
    return SubSets

For example, for the $[27,23,3]_9$ Hermitian code $H_{25}$ over $\mathbb{F}_9$ the length of the code is $n=27$. If we want to generate all the possible word positions for vectors of weight 3, we can run the command

```
GetPossibleWordPositions(27,3)

```


**Executable version below**

In [30]:
GetPossibleWordPositions(27, 3)

[{1, 2, 3},
 {1, 2, 4},
 {1, 2, 5},
 {1, 2, 6},
 {1, 2, 7},
 {8, 1, 2},
 {1, 2, 9},
 {1, 2, 10},
 {1, 2, 11},
 {1, 2, 12},
 {1, 2, 13},
 {1, 2, 14},
 {1, 2, 15},
 {16, 1, 2},
 {1, 2, 17},
 {1, 2, 18},
 {1, 2, 19},
 {1, 2, 20},
 {1, 2, 21},
 {1, 2, 22},
 {1, 2, 23},
 {24, 1, 2},
 {1, 2, 25},
 {1, 2, 26},
 {1, 2, 27},
 {1, 3, 4},
 {1, 3, 5},
 {1, 3, 6},
 {1, 3, 7},
 {8, 1, 3},
 {1, 3, 9},
 {1, 10, 3},
 {11, 1, 3},
 {1, 3, 12},
 {1, 3, 13},
 {1, 3, 14},
 {1, 3, 15},
 {16, 1, 3},
 {1, 3, 17},
 {1, 18, 3},
 {19, 1, 3},
 {1, 3, 20},
 {1, 3, 21},
 {1, 3, 22},
 {1, 3, 23},
 {24, 1, 3},
 {1, 3, 25},
 {1, 26, 3},
 {27, 1, 3},
 {1, 4, 5},
 {1, 4, 6},
 {1, 4, 7},
 {8, 1, 4},
 {1, 4, 9},
 {1, 10, 4},
 {1, 11, 4},
 {1, 4, 12},
 {1, 4, 13},
 {1, 4, 14},
 {1, 4, 15},
 {16, 1, 4},
 {1, 4, 17},
 {1, 18, 4},
 {1, 19, 4},
 {1, 4, 20},
 {1, 4, 21},
 {1, 4, 22},
 {1, 4, 23},
 {24, 1, 4},
 {1, 4, 25},
 {1, 26, 4},
 {1, 27, 4},
 {1, 5, 6},
 {1, 5, 7},
 {8, 1, 5},
 {1, 5, 9},
 {1, 10, 5},
 {1, 11, 5},
 {1, 12,

Similary, we want to, for all of these possible word positions, generate all possible (non-zero) coefficients these vectors could have. To that end, we define the following function:

In [31]:
def GetPossibleCoefficients(Field,weight):
    # First, get the vector space Field^weight, that is, vectors with 'weight' values living in 'Field'
    VS=VectorSpace(Field,weight)
    
    # Now, restrict your elemnts to only those having the desired weight
    Weighted_VS=[elem for elem in VS if elem.hamming_weight()==weight]
    
    # return the set of all possible (scaled) coefficients
    return Weighted_VS

For example, for the $[27,23,3]_9$ Hermitian code $H_{25}$ over $\mathbb{F}_9$, if we want to get the possible coefficients of a word of weight 3, we can run 

```
GetPossibleCoefficients(GF(9,'a'),3)
```

**Executable version below**

In [32]:
GetPossibleCoefficients(GF(9,'a'),3)

[(a, a, a),
 (a + 1, a, a),
 (2*a + 1, a, a),
 (2, a, a),
 (2*a, a, a),
 (2*a + 2, a, a),
 (a + 2, a, a),
 (1, a, a),
 (a, a + 1, a),
 (a + 1, a + 1, a),
 (2*a + 1, a + 1, a),
 (2, a + 1, a),
 (2*a, a + 1, a),
 (2*a + 2, a + 1, a),
 (a + 2, a + 1, a),
 (1, a + 1, a),
 (a, 2*a + 1, a),
 (a + 1, 2*a + 1, a),
 (2*a + 1, 2*a + 1, a),
 (2, 2*a + 1, a),
 (2*a, 2*a + 1, a),
 (2*a + 2, 2*a + 1, a),
 (a + 2, 2*a + 1, a),
 (1, 2*a + 1, a),
 (a, 2, a),
 (a + 1, 2, a),
 (2*a + 1, 2, a),
 (2, 2, a),
 (2*a, 2, a),
 (2*a + 2, 2, a),
 (a + 2, 2, a),
 (1, 2, a),
 (a, 2*a, a),
 (a + 1, 2*a, a),
 (2*a + 1, 2*a, a),
 (2, 2*a, a),
 (2*a, 2*a, a),
 (2*a + 2, 2*a, a),
 (a + 2, 2*a, a),
 (1, 2*a, a),
 (a, 2*a + 2, a),
 (a + 1, 2*a + 2, a),
 (2*a + 1, 2*a + 2, a),
 (2, 2*a + 2, a),
 (2*a, 2*a + 2, a),
 (2*a + 2, 2*a + 2, a),
 (a + 2, 2*a + 2, a),
 (1, 2*a + 2, a),
 (a, a + 2, a),
 (a + 1, a + 2, a),
 (2*a + 1, a + 2, a),
 (2, a + 2, a),
 (2*a, a + 2, a),
 (2*a + 2, a + 2, a),
 (a + 2, a + 2, a),
 (1, a + 2, a)

### Scaled Coefficents

Since we know that the syndrome is a linear operator, then for any vector $\mathbf{x} \in \mathbb{F}_{q^2}$ and any $\alpha \in \mathbb{F}_{q^2}^{*}$ that

$Syndrome(\alpha \cdot \mathbf{x})=\alpha \cdot Syndrome(\mathbf{x})$

Because of this, we can avoid some syndrome calculations by scaling the vector so that the first non-zero coefficient is equal to 1, since we can then obtain the syndrome of all scalar multiples by simply performing a scalar multiplication. 

To that end, we define the following function: 

In [33]:
def GetPossibleScaledCoefficients(Field,weight):
    # First, get the vector space Field^weight, that is, vectors with 'weight' values living in 'Field'
    VS=VectorSpace(Field,weight)
    
    # Now, restrict your elemnts to only those having the desired weight
    Weighted_VS=[elem for elem in VS if elem.hamming_weight()==weight]
    
    # Finally, we can scale the coefficients by setting the first coefficient to 1
    Scaled_Weighted_VS=[elem for elem in Weighted_VS if elem[0]==1]
    
    # return the set of all possible (scaled) coefficients
    return Scaled_Weighted_VS

For example, for the $[27,23,3]_9$ Hermitian code $H_{25}$ over $\mathbb{F}_9$, if we want to get the possible **(scaled)** coefficients of a word of weight 3, we can run 

```
GetPossibleScaledCoefficients(GF(9,'a'),3)
```

**Executable version below**

In [34]:
GetPossibleScaledCoefficients(GF(9,'a'),3)

[(1, a, a),
 (1, a + 1, a),
 (1, 2*a + 1, a),
 (1, 2, a),
 (1, 2*a, a),
 (1, 2*a + 2, a),
 (1, a + 2, a),
 (1, 1, a),
 (1, a, a + 1),
 (1, a + 1, a + 1),
 (1, 2*a + 1, a + 1),
 (1, 2, a + 1),
 (1, 2*a, a + 1),
 (1, 2*a + 2, a + 1),
 (1, a + 2, a + 1),
 (1, 1, a + 1),
 (1, a, 2*a + 1),
 (1, a + 1, 2*a + 1),
 (1, 2*a + 1, 2*a + 1),
 (1, 2, 2*a + 1),
 (1, 2*a, 2*a + 1),
 (1, 2*a + 2, 2*a + 1),
 (1, a + 2, 2*a + 1),
 (1, 1, 2*a + 1),
 (1, a, 2),
 (1, a + 1, 2),
 (1, 2*a + 1, 2),
 (1, 2, 2),
 (1, 2*a, 2),
 (1, 2*a + 2, 2),
 (1, a + 2, 2),
 (1, 1, 2),
 (1, a, 2*a),
 (1, a + 1, 2*a),
 (1, 2*a + 1, 2*a),
 (1, 2, 2*a),
 (1, 2*a, 2*a),
 (1, 2*a + 2, 2*a),
 (1, a + 2, 2*a),
 (1, 1, 2*a),
 (1, a, 2*a + 2),
 (1, a + 1, 2*a + 2),
 (1, 2*a + 1, 2*a + 2),
 (1, 2, 2*a + 2),
 (1, 2*a, 2*a + 2),
 (1, 2*a + 2, 2*a + 2),
 (1, a + 2, 2*a + 2),
 (1, 1, 2*a + 2),
 (1, a, a + 2),
 (1, a + 1, a + 2),
 (1, 2*a + 1, a + 2),
 (1, 2, a + 2),
 (1, 2*a, a + 2),
 (1, 2*a + 2, a + 2),
 (1, a + 2, a + 2),
 (1, 1, a + 2)

## Creating the syndrome table

Using these functions, we can create a table to store the vectors of low weight, represented by their non-zero positions and the respective coefficients, as well as the syndrome, and some additional information such as the code, when the syndrome was calculated, and wether or not the syndrome is zero.


In [35]:
def CreateEmptySyndromeTable(Code, weight,saveToFile=False, filename=""):
    
    # Set the column names we will use
    # These will store information about the Code parameters, and then the weight of the word, along 
    # with its non-zero positions and corresponding coefficients
    # we then use them to calculate the syndrome with the method described above
    # finally we save the timestamp and wether or not the syndrome is equal to 0
    colnames=['Code','Weight','Positions','Coefficients','SyndromeCalculated',
              'Syndrome','Timestamp', 'isZero?']
    
    #set the syndrome table using the column names
    SyndromeTable=pd.DataFrame(columns=colnames)
    
    
    # Extract the code parameters from 'Code'
    n=Code.length()
    k=Code.dimension()
    F=Code.base_field()
    q=len(F)
    CodeInfoString=f'[{n},{k}]_{q} code over {F}'
    
    # Now we will loop through weights 1 up to 'weight' to get the set of all possible postions and ccoefficients for each weight
    for w in range(1,weight+1):
        # Get the set of possible positions
        A=GetPossibleWordPositions(n, w)
        
        # Get the set of possible scaled coefficients
        B=GetPossibleScaledCoefficients(F,w)
        
        # In our testing, it ended up being faster setting up a cartesian product of positions and coeficients 
        # rather than looping through both lists
        C=cartesian_product([A,B])
        AA=[e[0] for e in C] # cartesian projections
        BB=[e[1] for e in C] # cartesian projections
        
        #sort the sets for the positions 
        AA=[sorted(e) for e in AA]
        
        # Now, for the weight w set up a temporary data frame to save the positions weight, 
        # and set up a space to get the syndrome and timestamp later
        X=pd.DataFrame(columns=colnames, index=range(len(C)))
        # save the positions
        X.Positions=AA
        # save the coefficients
        X.Coefficients=BB

        # save the code info
        X.Code=CodeInfoString
        # save the weight 
        X.Weight=w
        # and set SyndromeCalculated? to False
        X.SyndromeCalculated=False
        
        #Finally, concatenate the temporary table to the existing table
        SyndromeTable=pd.concat([SyndromeTable,X])
        
    # if saveToFile is True, check if the filename is not empty
    if(saveToFile):
        #if the filename is empty, print the error message and return the table
        if(filename==""):
            print("No filename provided. returning the syndorme table instead")
            return SyndromeTable
        else:
            # if the filename is not empty, dump it into the pickle file
            # TODO: implement try-catch
            pickle.dump(SyndromeTable,open(filename,'wb'))
        
    # finally, if saveToFile is set to False (default) just return the table    
    else:
        return SyndromeTable
    

For the $[27,23,3]_9$ Hermitian code $H_{25}$ over $\mathbb{F}_9$ going up to weight 3, we can create the table with the following code:

```
Code=HermitianCode(25,3)
EmptyTable=CreateEmptySyndromeTable(Code, 3,)
```

**Executable version below**

In [36]:
Code=HermitianCode(25,3)
EmptyTable=CreateEmptySyndromeTable(Code, 3,)

And this is what the table looks like

In [37]:
EmptyTable

,Code,Weight,Positions,Coefficients,SyndromeCalculated,Syndrome,Timestamp,isZero?
0,"[27,23]_9 code over Finite Field in a of size 3^2",1,[1],[1],False,NaN,NaN,NaN
1,"[27,23]_9 code over Finite Field in a of size 3^2",1,[2],[1],False,NaN,NaN,NaN
2,"[27,23]_9 code over Finite Field in a of size 3^2",1,[3],[1],False,NaN,NaN,NaN
3,"[27,23]_9 code over Finite Field in a of size 3^2",1,[4],[1],False,NaN,NaN,NaN
4,"[27,23]_9 code over Finite Field in a of size 3^2",1,[5],[1],False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
187195,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2, 1]",False,NaN,NaN,NaN
187196,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2*a, 1]",False,NaN,NaN,NaN
187197,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2*a + 2, 1]",False,NaN,NaN,NaN
187198,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, a + 2, 1]",False,NaN,NaN,NaN


Let's take a minute to examine a specific to row, to hone in on how to read the table

In [40]:
#you can change the index here. feel free to play around with this number 
i=100
EmptyTable.iloc[int(i):int(i+1)]

,Code,Weight,Positions,Coefficients,SyndromeCalculated,Syndrome,Timestamp,isZero?
73,"[27,23]_9 code over Finite Field in a of size 3^2",2,"[1, 11]","[1, a + 1]",False,NaN,NaN,NaN


Firstly, the first column, **Code** simply reminds us we are interested in the $[27,23]_9$ Hermitian code 

The columns **Weight**, **Positions** and **Coefficients** represent a particular vector $\mathbf{x}$ in $\mathbb{F}_{q^2}^{27}$. 

They state that $\mathbf{x}$ has weight 2, its non-zero positions are $1$ and $11$ and that the non-zero coefficients are $1$ and $a+1$ respectively. That is, 

$\mathbf{x}=[1,0,0,0,0,0,0,0,0,0,a+1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]$



In [41]:
def HermitianCodeFileString(m,q):
    return f'Syndrome_Table_Hermitian_Code(m={m},q={q}).p'

In [42]:
def FillSyndromeTable(filename,Code):
    
    #get the info for the code
    H=Code.parity_check_matrix()
    
    #load the syndrome table
    SyndromeTable=pickle.load(open(filename,'rb'))
    
    #get the current time
    checkpoint=dt.datetime.now()
    print(f"Loaded Syndrome Table at: {checkpoint}")
    
    #get the indices for the columns SyndromeCalculated, Syndrome and Checkpoint
    ind_syndrome_calculated = SyndromeTable.columns.get_loc('SyndromeCalculated')
    ind_syndrome            = SyndromeTable.columns.get_loc('Syndrome')
    ind_timestamp           = SyndromeTable.columns.get_loc('Timestamp')
    
    #Then, go through each row and calculte the syndrome, and save it.
    for i in range(0,SyndromeTable.shape[0]):
        
        #check if syndrome is not calculated, and calculate it, if needed
        if(not SyndromeTable.iloc[int(i),int(ind_syndrome_calculated)]):
             #save if 2 minutes have passed
            if((dt.datetime.now()-checkpoint).seconds>2*60):
                #update the checkpoint
                checkpoint=dt.datetime.now()
                
                #save the table
                pickle.dump(SyndromeTable,open(filename,'wb'))
                print(f'file: {filename} updated succesfully')
        
        
        
        # get the syndrome and coeficients
        coef=SyndromeTable.iloc[int(i)]['Coefficients']
        pos=SyndromeTable.iloc[int(i)]['Positions']
        
        syndrome=SyndromeCalculation(H, coef, pos)
        # print(f"syndrome calculated: {syndrome}")
        
        #set the syndrome
        SyndromeTable.iloc[int(i),int(ind_syndrome)]=syndrome
        
        #set SyndromeCalculated to True (to avoid computing again)
        SyndromeTable.iloc[int(i),int(ind_syndrome_calculated)]=True
        
        #put the timestamp for when this process is done 
        SyndromeTable.iloc[int(i),int(ind_timestamp)]=dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    
    # once all the syndromes are calculated get the checkpoint
    final_checkpoint=dt.datetime.now() 
    
    # finally, save the table
    pickle.dump(SyndromeTable,open(filename,'wb'))
    # and print a message 
    print(f'file: {filename} completed succesfully at {final_checkpoint}')
        
        

Now, we define a short function to make a string with the filename for the Hermitian code syndrome table. 

This is to have consistency in our naming conventions

The files will be named 'Syndrome_Table_Hermitian_Code(m='m',q='q').p

In [43]:
def HermitianCodeFileString(m,q):
    return f'Syndrome_Table_Hermitian_Code(m={m},q={q}).p'

In [44]:
m=25
q=3

HermitianCodeFileString(m,q)

'Syndrome_Table_Hermitian_Code(m=25,q=3).p'

## Putting it all together

**TODO: expand on the explanation for how to combine all the code to fill out the table**

### [27,23,3]_9 Hermitian Code (m=25)

First, we create the Hermitian code, and the filename we will use for the table

In [46]:
# set the parameters
m=25
q=3

# create the code
Code=HermitianCode(m,q)

# and the check matrix
H=Code.parity_check_matrix()







In [47]:
# and the filename
filename=HermitianCodeFileString(m,q)


In [36]:
# Now, create the empty syndrome table
CreateEmptySyndromeTable(Code, 3,True, filename)

In [48]:
SyndTable=pickle.load(open(filename,'rb'))

In [38]:
SyndTable

,Code,Weight,Positions,Coefficients,SyndromeCalculated,Syndrome,Timestamp,isZero?
0,"[27,23]_9 code over Finite Field in a of size 3^2",1,[1],[1],False,NaN,NaN,NaN
1,"[27,23]_9 code over Finite Field in a of size 3^2",1,[2],[1],False,NaN,NaN,NaN
2,"[27,23]_9 code over Finite Field in a of size 3^2",1,[3],[1],False,NaN,NaN,NaN
3,"[27,23]_9 code over Finite Field in a of size 3^2",1,[4],[1],False,NaN,NaN,NaN
4,"[27,23]_9 code over Finite Field in a of size 3^2",1,[5],[1],False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
187195,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2, 1]",False,NaN,NaN,NaN
187196,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2*a, 1]",False,NaN,NaN,NaN
187197,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2*a + 2, 1]",False,NaN,NaN,NaN
187198,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, a + 2, 1]",False,NaN,NaN,NaN


In [41]:
FillSyndromeTable(filename, Code)

Loaded Syndrome Table at: 2023-05-09 00:34:08.825318
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p updated succesfully
file: Syndrome_Table_Hermitian_Code(m=25,q=3).p completed succesfully at 2023-05-09 01:44:55.499423


Filling out the whole table is a computationally expensive process, and the size of the table grows exponentially. 

However, once it is computed, we can load it, and analyze the syndromes

In [49]:
FilledTable=pickle.load(open('Syndrome_Table_Hermitian_Code(m=25,q=3).p','rb'))
FilledTable

,Code,Weight,Positions,Coefficients,SyndromeCalculated,Syndrome,Timestamp,isZero?
0,"[27,23]_9 code over Finite Field in a of size 3^2",1,[1],[1],True,"[1, 0, 0, 0]",2023-05-09 00:34:08,NaN
1,"[27,23]_9 code over Finite Field in a of size 3^2",1,[2],[1],True,"[0, 1, 0, 0]",2023-05-09 00:34:08,NaN
2,"[27,23]_9 code over Finite Field in a of size 3^2",1,[3],[1],True,"[2, 2, 0, 0]",2023-05-09 00:34:08,NaN
3,"[27,23]_9 code over Finite Field in a of size 3^2",1,[4],[1],True,"[0, 0, 1, 0]",2023-05-09 00:34:08,NaN
4,"[27,23]_9 code over Finite Field in a of size 3^2",1,[5],[1],True,"[2, 1, 1, 0]",2023-05-09 00:34:08,NaN
...,...,...,...,...,...,...,...,...
187195,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2, 1]",True,"[2*a, 0, 1, a]",2023-05-09 01:44:55,NaN
187196,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2*a, 1]",True,"[2*a + 1, 0, 2*a + 2, a + 2]",2023-05-09 01:44:55,NaN
187197,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, 2*a + 2, 1]",True,"[1, 0, 2*a + 1, 2]",2023-05-09 01:44:55,NaN
187198,"[27,23]_9 code over Finite Field in a of size 3^2",3,"[25, 26, 27]","[1, a + 2, 1]",True,"[a + 2, 0, a + 1, 2*a + 1]",2023-05-09 01:44:55,NaN


## Syndrome Distribution

With the syndrome table calculated, we can easily get the number of unique syndromes, and compare with $(q^2){n-k}$. 

In [50]:
SyndromeDistribution=FilledTable.Syndrome.value_counts()

In [51]:
SyndromeDistribution

TypeError: mutable vectors are unhashable

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
  File "sage/modules/free_module_element.pyx", line 1099, in sage.modules.free_module_element.FreeModuleElement.__hash__ (build/cythonized/sage/modules/free_module_element.c:9398)
TypeError: mutable vectors are unhashable


[a + 2, 2*a + 2, 0, 0]          100
[a, 2*a + 1, 0, 0]              100
[2*a + 2, a + 2, 0, 0]          100
[a + 1, 2*a, 0, 0]              100
[2*a + 1, a, 0, 0]              100
                               ... 
[a + 1, 2*a + 2, 0, 0]            9
[0, 0, 0, 0]                      9
[a + 2, 2*a + 1, 0, 0]            9
[2*a + 1, a + 2, a + 1, 2*a]      7
[0, 2*a, 2, a + 2]                6
Name: Syndrome, Length: 6561, dtype: int64

In [54]:
num_unique=len(SyndromeDistribution)

In [55]:
num_unique

6561

Notice that this number is $6561=9^4$ And so this means that the covering radius $C(H_{25})$ over $\mathbb{F}_9$ is at most 3